# 뉴스 크롤링

In [ ]:
pip install selenium

In [ ]:
pip install webdriver-manager

In [ ]:
!apt-get update
!apt install chromium-chromedriver

In [84]:
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [111]:
# 기사 링크 크롤링
# 카테고리당 100개 = 5페이지

url_list = []   # url 저장 리스트
a_list = []     # a 태그 저장 리스트

for category in range(3, 4):     # 6
    for page in range(1, 2):  # 1, 6
        url = f'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1={100 + category}#&date=%2000:00:00&page={page}'
        browser.get(url)

        time.sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        a_list.extend(soup.select(".type06_headline dt+dt a"))
        a_list.extend(soup.select(".type06 dt+dt a"))

        print("카테고리 : ", 100 + category, "페이지 : ", page)

for a in a_list:  # 링크
    url_list.append(a["href"])

카테고리 :  103 페이지 :  1


In [ ]:
url_list

In [112]:
# 수집한 링크의 제목, 본문, 날짜 크롤링

title = []
content = []
date = []

for url in url_list:

      browser.get(url)

      time.sleep(1)

      soup = BeautifulSoup(browser.page_source, "html.parser")

      title.extend(soup.select("#title_area span"))
      content.extend(soup.select("#dic_area"))
      date.extend(soup.select("._ARTICLE_DATE_TIME"))

In [113]:
# 위에서 수집한 태그 속에서 내용만 뽑아서 담는다.

def clean(article):   # 전처리
  article = re.sub('\n','',article)
  article = re.sub('\t','',article)
  article = re.sub('\u200b','',article)
  article = re.sub('\xa0','',article)
  article = re.sub('([a-zA-Z])','',article)
  article = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',article)
  article = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',article)

  return article

title_list = []     # 제목
content_list = []   # 본문
date_list = []      # 날짜

for t in title:
  title_list.append(t.text)

for c in content:
  content_list.append(clean(c.text))

for d in date:
  date_list.append(d["data-date-time"])

In [114]:
# 데이터 프레임으로 변환

article_df = pd.DataFrame({"날짜" : date_list,
                           "제목" : title_list,
                           "본문" : content_list,
                           "링크" : url_list})

article_df

,날짜,제목,본문,링크
0,2023-10-27 20:01:01,속 더부룩할 때 콜라 대신 먹으면 좋은 ‘이 음료’,허브차는 뱃속에 가스가 차면서 생긴 복부 팽만 증상을 완화하고 위장관 근육을 부드럽...,https://n.news.naver.com/mnews/article/346/000...
1,2023-10-27 19:22:12,"[날씨] 주말 쾌청한 하늘…큰 일교차, 강원 영동 비 조금",앵커 주말인 내일은 전국이 대체로 맑겠습니다 낮동안 선선하지만 일교차가 크겠고 강원...,https://n.news.naver.com/mnews/article/422/000...
2,2023-10-27 19:03:01,수려한 경관과 누각의 조화… 죽서루·영남루 국보 된다 [뉴스 투데이],관동팔경 백미 삼척 죽서루’12세기 창건돼 조선 때 증축 거듭당대 시기별 건축적 특...,https://n.news.naver.com/mnews/article/022/000...
3,2023-10-27 19:01:01,정자 없는데 임신 가능? 남현희도 속은 ‘가짜 임신 테스트기’ 정체는…,해외 직구 플랫폼에서 가짜 임신 테스트기 판매가 이뤄지고 있었다사진알리익스프레스 캡...,https://n.news.naver.com/mnews/article/346/000...
4,2023-10-27 18:39:01,"[책마을] 하버드 85년 연구의 결과…""행복, 돈 아닌 '관계'에 있다""",세상에서 가장 긴 행복 탐구 보고서로버트 월딩거·마크 슐츠 지음박선령 옮김비즈니스북...,https://n.news.naver.com/mnews/article/015/000...
5,2023-10-27 18:31:01,"""반려견도 ADHD""...견주가 어떻게 키우냐에 달렸다",반려견을 키울 때 무조건적인 애정은 반려견을 문제견으로 만들 수 있고 주인이나 가족...,https://n.news.naver.com/mnews/article/296/000...
6,2023-10-27 18:27:01,"감리교, 제 35회 총회 입법의회 폐회…신대원 통합·여교역자 차별 금지 조항 눈길",기독교대한감리회 제35회 총회 입법의회가 26일 예정보다 하루 일찍 폐회했다 사진은...,https://n.news.naver.com/mnews/article/079/000...
7,2023-10-27 18:23:01,"[생활날씨] 주말 아침 쌀쌀, 큰 일교차…오후 한때 강원 영동 '비'",앵커 주말인 내일 전국이 맑겠지만 일교차가 커서 건강 관리에 유의하셔야겠습니다 강원...,https://n.news.naver.com/mnews/article/422/000...
8,2023-10-27 18:15:06,"기아, 올 영업익 벌써 9兆 넘어…영업이익률 글로벌車 '톱 티어'",3분기 매출·영업익 사상 최대고수익 늘고 친환경차 선전기아가 올해 13분기 누적 ...,https://n.news.naver.com/mnews/article/015/000...
9,2023-10-27 18:14:05,포드도 전기차 투자 축소…SK온에 '불똥',수요 감소 우려에 120억弗 축소노조 파업에 인건비 부담도 커져켄터키 배터리 2공장...,https://n.news.naver.com/mnews/article/015/000...


# 전처리

**KoNLPy**
- 한국어 정보처리를 위한 파이썬 패키지
- https://konlpy.org/ko/latest/api/konlpy.tag/#okt-class

**CountVectorizer**
- 카운트 기반 벡터화
- https://bkshin.tistory.com/entry/NLP-6-%EC%B9%B4%EC%9A%B4%ED%84%B0-%EA%B8%B0%EB%B0%98-%EB%B2%A1%ED%84%B0%ED%99%94CountVectorizer%EC%99%80-TF-IDF-%EB%B2%A1%ED%84%B0%ED%99%94

In [ ]:
pip install konlpy

In [118]:
from konlpy.tag import Okt

In [122]:
okt = Okt()
nouns_list = []

for content in article_df["본문"]:
  nouns_tmp = []

  content = clean(content)         # 전처리
  content = okt.nouns(content)     # 명사 추출

  for c in content:
    if len(c) != 1:
      nouns_tmp.append(c)
  nouns_list.append(nouns_tmp)
print(nouns_list)

[['허브차', '뱃속', '가스', '복부', '증상', '완화', '장관', '근육', '이완', '도움', '사진', '클립아트', '코리아', '과식', '탄산음료', '절실', '상태', '탄산음료', '청량감', '느낌', '때문', '그대로', '느낌', '실제', '소화', '불량', '해결', '도움', '탄산음료', '대신', '매실', '허브차', '매실', '소화', '사실', '매실', '구연산', '소화기', '해독', '작용', '연동', '운동', '조절', '미각', '자극', '도우', '피크', '르산', '성분', '위장', '유해', '효과', '위산', '분비', '또한', '촉진', '위산', '분비', '과도', '분비', '매실', '매실', '음료', '섭취', '소화액', '분비', '허브차', '또한', '추천', '허브차', '장관', '근육', '이완', '뱃속', '가스', '복부', '증상', '완화', '도움', '장관', '근육', '이완', '소화', '기능', '회복', '위장', '진정', '페퍼민트', '설사', '증상', '양배추', '위장', '보호', '소화', '채소', '방법', '양배추', '비타민', '점막', '호르몬', '분비', '위산', '여러', '자극', '물질', '로부터', '위벽', '보호', '설포', '성분', '헬리', '코박터', '파일', '로리', '제한', '탄수화물', '분해', '효소', '아밀라아제', '디아스타아제', '지방', '분해', '효소', '리파아제', '소화', '촉진', '항산화', '효과', '카탈', '제도', '함유', '양배추', '영양성분', '손실', '생으로', '디아스타아제', '껍질', '때문', '껍질', '특유', '때문', '생으로', '먹기', '강판', '무즙', '다소', '기능', '과일', '키위', '액티니딘', '단백질', '분해', '소화', '촉진', '도움', '육류', '콩류', '유제품',